In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# install

In [ ]:
!pip install -q konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 25.7 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 형태소 분석기
from konlpy.tag import Okt

# 모델
import gensim
from gensim.models import Word2Vec, FastText

# 한글설치

In [ ]:
# 한글 깨짐 현상 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
df = pd.read_csv("final_drop_duplicated_1page.csv")

# 텍스트 클리닝

In [ ]:
def clean_text(text):

    # 특수문자 제거 (한글,공백만 남김)
    text = re.sub(r'[^가-힣\s]', '', text)

    # 반복 문자 제거 (예: "!!" -> "!")
    text = re.sub(r'(.)\1+', r'\1', text)

    # 단일 자음 및 모음 제거
    single_korean_letters = r'\b[ㄱ-ㅎㅏ-ㅣ]\b'
    text = re.sub(single_korean_letters, '', text)

    # 외국어만 있는 텍스트 제거 (한국어가 포함되지 않은 텍스트)
    if not re.search(r'[가-힣]', text):
        return ''

    return text

In [ ]:
clean_text = df["review"].apply(lambda x: clean_text(x))

# 공백 review drop
clean_drop_text = clean_text[clean_text != ""].reset_index(drop = True)

In [ ]:
clean_drop_text

0        역사 스파오 셔츠는 그냥 기본으로 하나씩은 들고있어야 합니다 다른 색갈들도 색감 너...
1        이미지 체인지를 조금 해보고싶어서 항상 스트릿한 룩만 입다가 남친룩입어볼려고 구매해...
2        스파오 셔츠는 색상별로 가서 이번이 번째 입니다 저는 오버한 핏을 원했어서 라지가 ...
3                             색깔도 예쁘게 빠졌고 깔끔하고 단정하게 입기 좋아요
4        안녕하세요후기를 남기에 앞서 저는 키가 몸무게 인 대 남성입니다이 셔츠는 정말이지 ...
                               ...                        
94554    오버핏으로 입기 위해 큰 사이즈로 샀습니다 생각보다 두껍진 않았어요 무난한 무지티로...
94555                          넘 맘에 들어서 깔별로 모으고있어요 이너로 최고임
94556                              군더기 없이 기본에 충실한 아이템입니다 굳
94557                    옷도 짱하고 좋아요 그리고 옷 색빠짐도 별로 없는 것 같아요
94558                   이너 티셔츠 가성비 최강입니다 정핏으로 라지 사이즈 잘 맞아요
Name: review, Length: 94559, dtype: object

# Okt

In [ ]:
okt = Okt()

In [ ]:
def preprocess_text(text):
    # 텍스트 정규화
    # normalized_text = okt.normalize(text)
    # 텍스트 토큰화
    tokenized_text = okt.morphs(text)
    return tokenized_text

In [ ]:
okt_tokenized_reviews = [preprocess_text(review) for review in clean_drop_text]

In [ ]:
len(okt_tokenized_reviews)

94559

# 참고한 사전

In [ ]:
# 사전 정의

# 사전 단어
total = ['사이즈','사이즈하','사이즈감','임사이즈','여사이즈','사이즌','뻐요사이즈','싸이즈','핏', "핏을",'핏입니다',
      '핏감도','핏나','핏감이랑','핏처럼','핏입니당','핏이에요','핏이예요','핏이랑','핏이구요','핏이네요','핏감은',
      '핏입니','핏이라서','핏으로','옷핏','핏더','핏도','핏감','옷', '핏은','좋아요핏','요핏도',
      '좋아요핏도','상의','요핏','폼','폭','사이즈','옷','품','전체','품도','몸통','옷핏','상체','핏',
      '핏이구','핏이었','핏임','핏입', '핏입니','핏감','요핏','폼','핏이','크기','상품','상의','핏더']

chongjang = ['길이','기장','총장','궁디','궁딩','엉덩이','밑','밑단','위아래','끝단','밑기장',
             '밑단이랑','기장이','기장입니','밑단','밑','기장감','요기장','총길이','총장','총기장',
            '길이감','끝','기장','길이']

shoulder = ['어깨핏','어께','어깨','너비','어깡','골격','바디','가로','넓이','등판','통',
            '어깨깡패','어깨라','어깨넓','어깨쪽','어께','몸집','어깨핏','어깨선','어깨라인','어깡',
            '어깨','통','몸통','골격','넓이','너비','등판']

chest = ['가슴','둘레','바디','가로','넓이','통','둘레','몸집','가슴팍','통','몸통']

arm = ['소매','팔도','손목','손','팔다리','팔목','팔','팔꿈치','팔이','요팔',
       '당팔','팔길이','팔다리','팔길','팔소매','손목','팔목','소매통','손','팔','소매','팔기장']

small = ['작습니다','짧았어요','짧습니다','짧으면','짧','작았어요','짧다고','쪼여서','작았으면',
      '작아도','작으면','작음','작아진','좁다고',
      '짧아요','짧게','짧다는','좁다는','작네','작기는','달라붙는','작은데','짧은데','짧음',
      '작긴','짧네요','작게','작아서','작은','짧고','짧아','좁은','타이트','짧았지만','붙어서','짧긴',
      '작은것','작아요','짧은','짧아서','짧지만','좁아','좁고','작네요','작아','쪼이는','좁아요','크롭느낌',
      '크롭하','작고','숏','쫄려','작다고','작다','좁아서','작','크롭','크롭된','미니','크롭해',
      '길었으면','컸으면','좁게','짧지' ,'작지','크롭이','핏되',
      '숏하','사이즈업하','크롭하','숏한','크롭하긴','크롭이','크롭해','크롭느낌','크롭된','사이즈업','짝',
       '작','숏','짧긴','크롭','크롭한','타이트','크롭이라','작긴',
       '작','짧','좁']

big = ['큰','긴','커요','크고','크긴','넉넉하고','길어서','넉넉하게','작다는',
     '넉넉한','클','길고','큰데','크지','크지만','부해','크니깐','크더라구요','컸어도','넓음','큼직하게',
     '박시하네요','넉넉할','헐렁하게','큼직하니','컸음','큽니','긴데','길어','길게','넓어서','넓어','박시하고','컸어요',
     '박시','크다고','크다는','넓게','넓고','커용','길다는','크지도','큼','크다','길었어요','넉넉함','큰데도','덮는데',
     '덮이고','컸고','벙벙해서','크더라고요','컸을','크거나','크더라','컸었는데','컸어용','컸습니다','덮네요',
     '큼지막해서','덥는','박시하게','크네요','클까','넉넉하구','박시하','컸는데','넉넉하네요','크게','크니까','넉넉한데','길어요',
     '컸지만','헐렁한','크네','넉넉해','펑퍼짐한','덮어용','박시하니','헐렁해','넉넉하지''넉넉하구요','커','큽니다','넉넉하니',
     '크','길지','박시한데','덮어요','길긴','넉넉해요','넓은','덮히는', '넓긴','덮어줘서','덮은','넉넉해서','큼직해서','기네','낙낙',
     '흘러내리는','덮어서','벙벙한','커서','큰것','박시핏','오퍼핏','낙낙해','여유핏','널널함','길','덮어','넉넉합니다','크면',
     '덮고','가오리','루즈핏','펑퍼짐','길었지만','헐렁하고','남아','아방핏','박스핏','널널','오바핏','덮습니다','넉넉히',
     '헐렁해서','접어도','접어','접어서','널','넓어요','덮고도','오버','와이드','낭낭','헐렁','접으면','짧았으면','접고',
     '버핏','접어야','잡아먹힌','수선해서','오버핏','오버사이즈','오버사이징해서','낙낙하다','낙낙한데','낙낙해요','오버핏이예요','오버핏입니당',
       '요박시하','컸어용','낙낙합니','어벙벙하','와이드핏','크니깐','크네요옷','널널해요','오버핏나와요','널럴하',
       '요넉넉하','엄청큼','여유핏','오버느낌','아방핏','오버핏은','넓긴','넉넉함','입니다오버핏','큽니다오버핏',
       '큽니다제','큽니당','오버해요','요박시한','헐렁하긴','헐렁거리','아방방한','아방방','박시합니','박시했다',
       '빅사이즈','받았습니다오버핏','빡시하','루즈핏으로','낭낭한','오버핏이긴','박스핏','널널함','어벙벙',
       '요넉넉한','오퍼핏','오버핏이구요','낙낙해','낭낭해서','넉넉하구','넉넉한데','아방해','오버핏이라서',
       '낭낭하','박시해','커용','넉넉해','펑퍼짐', '널널','좋아요오버핏','어벙벙한','널널하','오바핏',
       '오버하지','오버핏하','박시합니다','오버핏되','박시함','버핏','박시한','낙낙하니','박시해요','크네용',
       '낙낙해서','큼직해서','루즈해','느슨','널널합니다','아방아','아방아방','루즈핏이','박시하','세미오버',
       '세미오버핏','루즈해서','오버핏이라','루즈핏이라','오버핏이네요','오버핏입니','어벙','오버해','박시한데',
       '아방','오버핏이에요','예뻐요오버핏','큼지막','박시핏','오버해서','루즈한','오버핏인데','넉넉하구요','박시할',
       '큼직','오버한','오버핏이','낙낙하','아방하','롱','오버핏으로','낙낙한','오버핏입니다','박시해서','와이드',
       '요오버핏','넉넉해서','큽니','오버','오버하','루즈','루즈핏','박시','넉넉','길','크긴','아방한','널널한',
       '헐렁','길긴','와이드하','아방해서','세미오버사이즈',
       '크','길','널','덮','넓','벙벙하','덮이','덮히','커다랗','덥히','넉넉해지','뒤집', '넉넉',

       ]


# word2vec

In [ ]:
okt_word2vec = Word2Vec(okt_tokenized_reviews, vector_size = 200, window = 1, min_count = 1, sg = 1)

In [ ]:
okt_word2vec.wv.most_similar("총장", topn=30)

In [ ]:
similar_words = okt_word2vec.wv.most_similar("총장", topn=30)
words_only = [word for word, similarity in similar_words]

# 단어 출력
for word in words_only:
    print(word)